In [454]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
processed_data = np.load('Full_array.npy')

from tensorflow import keras

import pandas as pd

#The size of the dataset is 10,000; but I only filled the first 9166 with values, the rest are zeros. (shouldn't be counted)
number = 9166
processed_data = processed_data[0:number]

np.random.seed(1)
tf.set_random_seed(1)

In [455]:
#we need to push accuracy to over 90%

#idea number 1: run a set of high percentage accuracy classifications through the NN (nope)

#idea number 2: clean the training set by running through and deleting the bad examples from my data aquisition, were going to 
#set all arrays of zero to be class A. If I feed this NN nothing, I want it to tell me it is odd. (yup)

#first lets see what doing number 2 will get us.

#augment data set manually (worked)

#change LR (Adam is adaptive: so no)

In [456]:
#we want to normalize our iamge data to go from 0 to 1
processed_data = processed_data.reshape(np.size(processed_data))
processed_data = (processed_data - min(processed_data))/ (max(processed_data) - min(processed_data))
processed_data = processed_data.reshape(number,28,28)
#I actually am not sure how it is scaled. however, some of the pixels are negative for some reason???

#also, this did not help.

In [457]:
#hyperparameters:
HP_allowed_confidence = 0.98
test_train_split = 0.85
batch_size = 256
epoch_number = 15
learning_rate = 0.001
dropout_rate = 0.3
CNL1_filters = 32
CNL1_kernal_size = 3
MPL1_pool_size= (2,2)
MPL1_strides = 2
CNL2_filters = 64
CNL2_kernal_size = 3
MPL2_pool_size = (2,2)
MPL2_strides = 2

#I have a non-round number of examples
train_split_indice = int(np.round(test_train_split*number))

In [458]:
#Load in my dataset of targets, targets are strings labels under the name "Class"
galaxyzoo = pd.read_csv("zoo2MainSpecz.csv/zoo2MainSpecz.csv", usecols=[2,3,4,8,15,21,27], nrows=number)
#galaxyzoo = pd.read_csv("zoo2MainSpecz.csv/zoo2MainSpecz.csv", nrows=number)
Class = galaxyzoo["gz2class"].values
RA = galaxyzoo['ra'].values
DEC = galaxyzoo['dec'].values
Spiral = galaxyzoo['t01_smooth_or_features_a02_features_or_disk_debiased'].values
Elliptical = galaxyzoo['t01_smooth_or_features_a01_smooth_debiased'].values
Anythingelse = galaxyzoo['t01_smooth_or_features_a03_star_or_artifact_debiased'].values

In [459]:
#I want to take the first character of the Class string and interpret as a integer, ala MNIST example code
dictionary = {'A':int(2),'E':int(1),'S':int(0)}

In [460]:
#resave using my dictionary
target = np.empty((len(Class)))
for i in range(len(Class)):
    target[i] = int(dictionary[Class[i][0]])

In [461]:
#go through, find the arrays of zero, set that target to 'A' = 2
for i in range(len(target)):
    if processed_data[i].any() == 0:
        target[i] = 2

In [462]:
#split my data between training and test sets
train_target = target[0:train_split_indice]
test_target = target[train_split_indice:number]
train_images = processed_data[0:train_split_indice]
test_images = processed_data[train_split_indice:number]

In [463]:
#Create 3 more images from each of my training and testing images that is each image but flipped 90 degrees...
train_images_flip1 = np.flip(train_images,1)
train_images_flip2 = np.flip(train_images,2)
train_images_flip3 = np.flip(train_images_flip1,2)

flipped_img = np.append(train_images, train_images_flip1, 0)
flipped_img = np.append(flipped_img, train_images_flip2, 0)
flipped_img = np.append(flipped_img, train_images_flip3, 0)

flipped_tar = np.append(train_target,train_target, 0)
flipped_tar = np.append(flipped_tar,train_target, 0)
flipped_tar = np.append(flipped_tar,train_target, 0)

#test_images_flip1 = np.flip(test_images,1)
#test_images_flip2 = np.flip(test_images,2)
#test_images_flip3 = np.flip(test_images_flip1,2)

#test_images = np.append(test_images, test_images_flip1,0)
#test_images = np.append(test_images, test_images_flip2,0)
#test_images = np.append(test_images, test_images_flip3,0)

#test_target = np.append(test_target,test_target,0)
#test_target = np.append(test_target,test_target,0)

print(np.shape(flipped_tar))
print(np.shape(flipped_img))
print(np.size(train_target)*4)

print(np.shape(test_target))

(31164,)
(31164, 28, 28)
31164
(1375,)


In [464]:
train_images = flipped_img
train_target = flipped_tar

In [465]:
np.shape(train_images[0])

(28, 28)

In [466]:
from PIL import Image
#rotate each image in the target_images by a random amount
rotations_array = np.empty((31164,28,28))
for i in range(np.shape(train_images)[0]):
    degree = np.random.uniform(-45,45,1)
    img = Image.fromarray(train_images[i])
    rot = img.rotate(degree)
    rot = np.asarray(rot)
    rotations_array[i] = rot
#now append rotations_array to train_images
train_images = np.append(train_images, rotations_array,0)
#now append the class labels to the train_target array
train_target = np.append(train_target,train_target)


#every time we run this segment, we increase our training set by a factor of 2. this means it will train in 120 * number of times
# times 20 epochs = a long time.

In [467]:
from cv2 import resize

In [468]:
def cv2_clipped_zoom(img, zoom_factor):
    """
    Center zoom in/out of the given image and returning an enlarged/shrinked view of 
    the image without changing dimensions
    Args:
        img : Image array
        zoom_factor : amount of zoom as a ratio (0 to Inf)
    """
    height, width = img.shape[:2] # It's also the final desired shape
    new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)

    ### Crop only the part that will remain in the result (more efficient)
    # Centered bbox of the final desired size in resized (larger/smaller) image coordinates
    y1, x1 = max(0, new_height - height) // 2, max(0, new_width - width) // 2
    y2, x2 = y1 + height, x1 + width
    bbox = np.array([y1,x1,y2,x2])
    # Map back to original image coordinates
    bbox = (bbox / zoom_factor).astype(np.int)
    y1, x1, y2, x2 = bbox
    cropped_img = img[y1:y2, x1:x2]

    # Handle padding when downscaling
    resize_height, resize_width = min(new_height, height), min(new_width, width)
    pad_height1, pad_width1 = (height - resize_height) // 2, (width - resize_width) //2
    pad_height2, pad_width2 = (height - resize_height) - pad_height1, (width - resize_width) - pad_width1
    pad_spec = [(pad_height1, pad_height2), (pad_width1, pad_width2)] + [(0,0)] * (img.ndim - 2)

    result = resize(cropped_img, (resize_width, resize_height))
    result = np.pad(result, pad_spec, mode='constant')
    assert result.shape[0] == height and result.shape[1] == width
    return result

In [469]:
#increase by a factor of two again, like above
rotations_array = np.empty((31164*2,28,28))
for i in range(np.shape(train_images)[0]):
    degree = np.random.uniform(-45,45,1)
    img = Image.fromarray(train_images[i])
    rot = img.rotate(degree)
    zoom_factor = np.random.uniform(0.7,1.3,1)
    rot = cv2_clipped_zoom(np.asarray(rot),zoom_factor)
    #rot = rot.resize()
    #rot = np.asarray(rot)
    rotations_array[i] = rot
#now append rotations_array to train_images
train_images = np.append(train_images, rotations_array,0)
#now append the class labels to the train_target array
train_target = np.append(train_target,train_target)

In [470]:
#we want to form a subset of our training examples that are probabilities that they are sorted correctly greater than %90
#we will then train this dataset additional times.
"""
confidence = list()
HP_indice = list()
for i in range(len(train_images)):
    if train_target[i] == 0:
        confidence.append(Spiral[i])
    if train_target[i] == 1:
        confidence.append(Elliptical[i])
    if train_target[i] == 2:
        confidence.append(Anythingelse[i])
    if confidence[i] >= HP_allowed_confidence:
        HP_indice.append(i)
train_weights = np.asarray(confidence)
HP_target = np.empty(len(HP_indice))
HP_images = np.empty((len(HP_indice),28,28))
for i in range(len(HP_indice)):
    HP_target[i] = train_target[HP_indice[i]]
    HP_images[i] = train_images[HP_indice[i]]

#print(len(HP_indice))
HP_images = HP_images.reshape(len(HP_indice),28,28,1)
train_images = train_images.reshape(train_split_indice,28,28,1)
test_images = test_images.reshape(train_split_indice-number,28,28,1)
print(np.shape(train_images))
print(np.shape(HP_images))
"""

#I need this bottom part to come out, but delete it if we wanted the HP set again...
train_images = train_images.reshape(train_split_indice*4*2*2,28,28,1)
test_images = test_images.reshape((train_split_indice-number)*4*2*2,28,28,1)

In [471]:
#Next Idea: I want to augment my images to artificially create more data: technically, I should be allowed to do any
#amount of flips and rotations because of the cosmological principle

#if that doesn't work, I will just have to get more data. or add more layers...

#Note: using this generator decreased performance, I dont know how it works. I do know how my manuel offline flips and rotations
#work. And, those offline incresed performance.

"""
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=45,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')
"""

"\nfrom keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img\n\ndatagen = ImageDataGenerator(\n        rotation_range=45,\n        #width_shift_range=0.2,\n        #height_shift_range=0.2,\n        #zoom_range=0.2,\n        horizontal_flip=True,\n        vertical_flip=True,\n        fill_mode='nearest')\n"

In [472]:
from sklearn.utils import shuffle
#X=[1,2,3]
#y = ['one', 'two', 'three']
#X, y = shuffle(X, y, random_state=0)
#print(X)
#print(y)
#train_images, train_target = shuffle(train_images, train_target, random_state=1)


In [473]:

#define my model, using a CNN with 2 Convolutional layers, 2 max pool layers, 1 dense layer, 1 drop out layer, and another dense layer 
def create_model(dropout_rate=dropout_rate, learning_rate=learning_rate):
    
    model = keras.Sequential([])
    model.add(keras.layers.Conv2D(input_shape=(28,28,1),filters=CNL1_filters,kernel_size=CNL1_kernal_size,padding="same",activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=MPL1_pool_size, strides=MPL1_strides))
    model.add(keras.layers.Conv2D(filters=CNL2_filters,kernel_size=CNL2_kernal_size,padding="same",activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=MPL2_pool_size,strides=MPL2_strides))
    #model.add(keras.layers.Dropout(rate=0.25))
    #model.add(keras.layers.Conv2D(filters=128,kernel_size=CNL2_kernal_size,padding="same",activation=tf.nn.relu))
    #model.add(keras.layers.MaxPool2D(pool_size=MPL2_pool_size,strides=MPL2_strides))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=1024,activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=dropout_rate))
    model.add(keras.layers.Dense(units=3,activation=tf.nn.softmax))
    adam = tf.train.AdamOptimizer(learning_rate = learning_rate)
    model.compile(optimizer=adam, loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return(model)



In [474]:
#Fit the model with the high probability first, 2-3 epochs increased accuracy just slightly.
#model=create_model(dropout_rate=dropout_rate, learning_rate=learning_rate)
#model.fit(HP_images, HP_target, epochs=2, batch_size=batch_size, verbose=1, shuffle=True)

In [475]:
#model.summary()
#test_loss, test_acc = model.evaluate(test_images.reshape(number - train_split_indice,28,28,1), test_target)
#print('Test accuracy:', test_acc)

In [481]:
#lets try training one epoch at a time and seeing what epoch when we maximize our accuracy; I fear overtraining

#OTHER than that, my next idea is to change the architecture.... eventually I will need to build a RESnet anyways
#model = create_model(dropout_rate=dropout_rate, learning_rate = learning_rate)
#I want to only present a few images now for each epoch, this may help with overfitting
model.fit(train_images, train_target, epochs=epoch_number, batch_size=batch_size, verbose=1, shuffle=True)
#for i in range(epoch_number):
#    model.fit(train_images[0:train_split_indice], train_target[0:train_split_indice], epochs=1, batch_size=batch_size, verbose=1, shuffle=True)
#    test_loss, test_acc = model.evaluate(test_images, test_target)
#    print('Test accuracy:', test_acc)
#overtrained at epoch 18?

Epoch 1/15
124656/124656 [==============================] - 245s 2ms/step - loss: 0.3675 - acc: 0.8387
Epoch 2/15
124656/124656 [==============================] - 245s 2ms/step - loss: 0.3634 - acc: 0.8396
Epoch 3/15
124656/124656 [==============================] - 243s 2ms/step - loss: 0.3576 - acc: 0.8427
Epoch 4/15
124656/124656 [==============================] - 245s 2ms/step - loss: 0.3528 - acc: 0.8458
Epoch 5/15
124656/124656 [==============================] - 243s 2ms/step - loss: 0.3484 - acc: 0.8475
Epoch 6/15
124656/124656 [==============================] - 242s 2ms/step - loss: 0.3422 - acc: 0.8498
Epoch 7/15
124656/124656 [==============================] - 244s 2ms/step - loss: 0.3384 - acc: 0.8504
Epoch 8/15
124656/124656 [==============================] - 246s 2ms/step - loss: 0.3316 - acc: 0.8547
Epoch 9/15
124656/124656 [==============================] - 243s 2ms/step - loss: 0.3276 - acc: 0.8566
Epoch 10/15
124656/124656 [==============================] - 249s 2ms/ste

In [477]:
#model = create_model(dropout_rate=dropout_rate, learning_rate = learning_rate)
#model.fit(train_images, train_target, epochs=1, batch_size=batch_size, verbose=1, shuffle=True

In [480]:
#model.summary()
test_loss, test_acc = model.evaluate(test_images, test_target)
print('Test accuracy:', test_acc)

#0.848 with the zeros fixed
#0.850 with the HP dataset implemented

#0.822 with augmentation, rescaling
#rescaling seems to worsen my performance; however: why are some of my pixel values negative?

#0.842 with 50 epochs: doesn't seem to be over fitting, perhaps the data augmentation just takes more time to train???

#0.852 with 75 epochs, maybe we need to decrease learning rate past 50???, using generator

#0.857 w/ 75 epochs, using generator

#0.864 with 20 epochs, using offline flips before the data set. dropout = 0.3, no HP set. perhaps we can train for 15 epochs with
#LR  0.001 as usual, then train 5 epochs with LR decreased by factor of 10? 

#I dont think changing my LR will help, it appears Adam already adapts LR on the fly for the situation.
#0.857 w/ droput rate increased to 0.3, will decrease it again... 

#0.864 with 15 epochs, dropout_rate = 0.25, with 124656 images after making two copies of flips and stuff.

#0.852 with 20 epcohs, dropout rate = 0.5, LR = 0.004, batch_size = 128. The accuracy did not seem over trained by the end of the 
#20 epochs, therefore I will run it through another 20 to see what happens

#result over trained...

#CANT PUSH THE GODDAMN NETWORK OVER 0.864!!!! AHHH! I have implemented a zoom, stochastic methods of presenting images

#Maybe just rescale the damn images???

#0.840 after 15 epochs, flips, zooms, and rotations, and rescaled images from 0 to 1.

1375/1375 [==============================] - 2s 1ms/step
Test accuracy: 0.8407272728139704
